In [0]:
schema_teste = "main.default"

In [0]:
# Create a DataFrame with sample data
data = [
    ("A", "2025-07-08", 100),
    ("A", "2025-07-09", 150),
    ("A", "2025-07-10", 200),
    ("B", "2025-07-08", 110),
    ("B", "2025-07-09", 160),
    ("B", "2025-07-10", 210),
    ("C", "2025-07-08", 120),
    ("C", "2025-07-09", 170),
    ("C", "2025-07-10", 220)
]

columns = ["uf", "date", "amount"]

nome_tabela = f"{schema_teste}.sales"
nome_func = f"{schema_teste}.sales_func"
df = spark.createDataFrame(data, columns)

# Create the table in 'main.default' and insert the data
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(nome_tabela)

# Display the table to verify
display(spark.table(nome_tabela))

In [0]:
%sql
SELECT
    is_account_group_member("TEST_GROUP_A"),
    is_account_group_member("TEST_GROUP_B"),
    is_account_group_member("TEST_GROUP_C")

In [0]:
spark.sql(f"""
SELECT 
    * 
FROM {nome_tabela}
WHERE
    (is_account_group_member("TEST_GROUP_A") AND uf = "A")
    OR (is_account_group_member("TEST_GROUP_B") AND uf = "B")
    OR (is_account_group_member("TEST_GROUP_C") AND uf = "C")
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION {nome_func}(uf STRING)
RETURN
    (is_account_group_member("TEST_GROUP_A") AND uf = "A")
    OR (is_account_group_member("TEST_GROUP_B") AND uf = "B")
    OR (is_account_group_member("TEST_GROUP_C") AND uf = "C")
;
""")

In [0]:
spark.sql(f"ALTER TABLE {table_name} DROP ROW FILTER;")
spark.sql(f"ALTER TABLE {table_name} SET ROW FILTER {nome_func} ON (uf);")